In [1]:
import nltk 
nltk.download('all')   # no need to download all packages: 'stem' would be enough.
from nltk.stem import WordNetLemmatizer
import re    # regular expression
!pip install numpy
import numpy as np
import pandas as pd
from pprint import pprint  # prettier print
import gensim       # Gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

# disable any gensim warnings
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')


nk to
[nltk_data]    |     C:\Users\ku-team\AppData\Roaming\nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Downloading package dolch to C:\Users\ku-
[nltk_data]    |     team\AppData\Roaming\nltk_data...
[nltk_data]    |   Package dolch is already up-to-date!
[nltk_data]    | Downloading package europarl_raw to C:\Users\ku-
[nltk_data]    |     team\AppData\Roaming\nltk_data...
[nltk_data]    |   Package europarl_raw is already up-to-date!
[nltk_data]    | Downloading package floresta to C:\Users\ku-
[nltk_data]    |     team\AppData\Roaming\nltk_data...
[nltk_data]    |   Package floresta is already up-to-date!
[nltk_data]    | Downloading package framenet_v15 to C:\Users\ku-
[nltk_data]    |     team\AppData\Roaming\nltk_data...
[nltk_data]    |   Package framenet_v15 is already up-to-date!
[nltk_data]    | Downloading package framenet_v17 to C:\Users\ku-
[nltk_data]    |     team\AppData\Roaming\nltk_data...
[nltk_data]    |   Pac

In [3]:
# load the save the english comments corpus into a pickle file as a list
pickle_in = open("english_comments.pickle","rb")
english_comments = pickle.load(pickle_in)    # loads the list of comments
pprint(english_comments[:1])

['Our stay with Marcus in Bristol was fantastic in every way! He was a great '
 'host - picking us up at the bus stop, recommending places to try, leaving '
 'plenty of pastries and other breakfast items to enjoy in the morning. The '
 'flat itself was modern, bright, clean and spacious - and best of all, right '
 "on Bristol's lovely harbourside. We will definitely stay again next time "
 "we're in Bristol - thanks again Marcus!"]


In [4]:
# put the list into dataframe
df = pd.DataFrame(english_comments,columns=['comments'])
df.head(5)

,comments
0,Our stay with Marcus in Bristol was fantastic ...
1,"Marcus is a brilliant, warm and friendly host...."
2,My mum Angela and I have stayed at Marcus' ama...
3,Marcus was an exceptional host. I only stayed ...
4,"Marcus was welcoming, easy going and very help..."


In [7]:
# tokenize sentences and remove panctuations and emojis
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [8]:
tokenized_texts = list(sent_to_words(english_comments))
print(tokenized_texts[:2])


[['our', 'stay', 'with', 'marcus', 'in', 'bristol', 'was', 'fantastic', 'in', 'every', 'way', 'he', 'was', 'great', 'host', 'picking', 'us', 'up', 'at', 'the', 'bus', 'stop', 'recommending', 'places', 'to', 'try', 'leaving', 'plenty', 'of', 'pastries', 'and', 'other', 'breakfast', 'items', 'to', 'enjoy', 'in', 'the', 'morning', 'the', 'flat', 'itself', 'was', 'modern', 'bright', 'clean', 'and', 'spacious', 'and', 'best', 'of', 'all', 'right', 'on', 'bristol', 'lovely', 'harbourside', 'we', 'will', 'definitely', 'stay', 'again', 'next', 'time', 'we', 're', 'in', 'bristol', 'thanks', 'again', 'marcus'], ['marcus', 'is', 'brilliant', 'warm', 'and', 'friendly', 'host', 'he', 'picked', 'us', 'up', 'from', 'the', 'railway', 'station', 'he', 'took', 'anne', 'to', 'the', 'doctor', 'and', 'drove', 'us', 'around', 'wherever', 'we', 'needed', 'to', 'go', 'in', 'bristol', 'and', 'dropped', 'us', 'back', 'at', 'the', 'railway', 'station', 'when', 'we', 'were', 'leaving', 'his', 'flat', 'is', 'very'

In [9]:
# see the most common and most rare words
from collections import  Counter    # dictionary collection
def get_words_frequency(texts):
    counter = Counter()
    for  doc in texts:
        for word in doc:
            counter[word]+=1
    return counter

word_freq = get_words_frequency(tokenized_texts)
pprint(sorted(word_freq))  # @ the end of the printed texts, still chinise words in there.

',
 'univ',
 'univerisity',
 'univeristy',
 'universal',
 'universally',
 'universe',
 'universities',
 'universitt',
 'university',
 'universiy',
 'unkempt',
 'unkept',
 'unknown',
 'unlabeled',
 'unleashed',
 'unless',
 'unlike',
 'unlikely',
 'unlimited',
 'unlit',
 'unload',
 'unloaded',
 'unloading',
 'unlock',
 'unlockable',
 'unlocked',
 'unlocking',
 'unloved',
 'unlovely',
 'unluckely',
 'unlucky',
 'unmade',
 'unmanaged',
 'unmarked',
 'unmatched',
 'unmetered',
 'unmissable',
 'unmopped',
 'unnecessarily',
 'unnecessary',
 'unneeded',
 'unnerved',
 'unnerving',
 'unnoticed',
 'unnumbered',
 'uno',
 'unobstrusive',
 'unobtrusive',
 'unobtrusively',
 'unoccupied',
 'unopen',
 'unopened',
 'unorganised',
 'unos',
 'unpack',
 'unpacking',
 'unparalleled',
 'unpersonally',
 'unpictured',
 'unplanned',
 'unpleasant',
 'unpleasantly',
 'unplugged',
 'unpractical',
 'unprepared',
 'unprepossessing',
 'unproblematic',
 'unproductive',
 'unprofessional',
 'unquestionably',
 'unravel',

In [10]:
# to filter chinese words or get only english words
def get_english_words(documents):
    ''' filters out any non-english words'''
    english_words = [[ w for w in doc if re.match(r'[A-Z, a-z]', w)] for doc in documents]
    return english_words


fitered_english_words = get_english_words(tokenized_texts)
pprint(fitered_english_words[:2])

[['our',
  'stay',
  'with',
  'marcus',
  'in',
  'bristol',
  'was',
  'fantastic',
  'in',
  'every',
  'way',
  'he',
  'was',
  'great',
  'host',
  'picking',
  'us',
  'up',
  'at',
  'the',
  'bus',
  'stop',
  'recommending',
  'places',
  'to',
  'try',
  'leaving',
  'plenty',
  'of',
  'pastries',
  'and',
  'other',
  'breakfast',
  'items',
  'to',
  'enjoy',
  'in',
  'the',
  'morning',
  'the',
  'flat',
  'itself',
  'was',
  'modern',
  'bright',
  'clean',
  'and',
  'spacious',
  'and',
  'best',
  'of',
  'all',
  'right',
  'on',
  'bristol',
  'lovely',
  'harbourside',
  'we',
  'will',
  'definitely',
  'stay',
  'again',
  'next',
  'time',
  'we',
  're',
  'in',
  'bristol',
  'thanks',
  'again',
  'marcus'],
 ['marcus',
  'is',
  'brilliant',
  'warm',
  'and',
  'friendly',
  'host',
  'he',
  'picked',
  'us',
  'up',
  'from',
  'the',
  'railway',
  'station',
  'he',
  'took',
  'anne',
  'to',
  'the',
  'doctor',
  'and',
  'drove',
  'us',
  'arou

In [11]:
# see words and their frequency
words_freq=  get_words_frequency(fitered_english_words)
pprint(sorted(words_freq)) #  chinise words are gone.

y',
 'unbeatable',
 'unbeknown',
 'unbeknownst',
 'unbelievable',
 'unbelievably',
 'unbelieved',
 'unblock',
 'unbooked',
 'unbreathable',
 'unbuild',
 'uncanny',
 'uncared',
 'uncaring',
 'uncertain',
 'uncertainty',
 'unchanged',
 'uncle',
 'unclean',
 'uncleaned',
 'uncleanliness',
 'unclear',
 'uncluttered',
 'uncomfortable',
 'uncomfortablewe',
 'uncomfortably',
 'uncomfy',
 'uncomlicated',
 'uncommercial',
 'uncommon',
 'uncompliated',
 'uncomplicated',
 'unconcerned',
 'unconfortable',
 'uncontactable',
 'unconvenient',
 'unconventional',
 'unconverted',
 'unconvinced',
 'uncooperative',
 'uncovered',
 'uncreased',
 'und',
 'undecipherable',
 'undeniable',
 'undeniably',
 'under',
 'underbed',
 'undercharging',
 'undercover',
 'undercroft',
 'underestimate',
 'underestimated',
 'underfall',
 'underfill',
 'underfloor',
 'underfoot',
 'undergoing',
 'undergrad',
 'underground',
 'underline',
 'underlined',
 'underneath',
 'underpinned',
 'underpriced',
 'underrated',
 'undersell

In [12]:
# Check the most common or frequent words
most_common_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
most_common_words[:350]   # first 20 words

[('and', 186280),
 ('the', 154077),
 ('to', 93356),
 ('was', 73952),
 ('in', 64651),
 ('is', 63608),
 ('very', 59606),
 ('great', 49954),
 ('for', 47042),
 ('stay', 43553),
 ('we', 42815),
 ('of', 41167),
 ('with', 37414),
 ('place', 32685),
 ('it', 32402),
 ('location', 30508),
 ('lovely', 30187),
 ('you', 26730),
 ('bristol', 26587),
 ('clean', 25628),
 ('room', 22558),
 ('would', 21626),
 ('host', 20727),
 ('really', 19666),
 ('house', 19121),
 ('nice', 18825),
 ('had', 18530),
 ('comfortable', 18213),
 ('recommend', 18204),
 ('at', 17680),
 ('as', 17458),
 ('good', 16679),
 ('this', 16439),
 ('were', 16193),
 ('flat', 15887),
 ('again', 14698),
 ('our', 14624),
 ('from', 14350),
 ('on', 13921),
 ('but', 13856),
 ('us', 13653),
 ('so', 13059),
 ('home', 13057),
 ('everything', 12855),
 ('there', 12672),
 ('all', 12538),
 ('perfect', 12394),
 ('well', 12204),
 ('my', 11955),
 ('apartment', 11907),
 ('definitely', 11705),
 ('are', 11510),
 ('city', 11453),
 ('easy', 11165),
 ('friendl

In [13]:
# see the rare words 
rare_words = sorted(word_freq.items(), key=lambda x: x[1])   # NB: word_freq is a dictionary colloction
pprint(rare_words[:100])  # first 100 rare words

[('chauffeur', 1),
 ('ireland', 1),
 ('foreward', 1),
 ('boatload', 1),
 ('markus', 1),
 ('cellular', 1),
 ('hovered', 1),
 ('saunter', 1),
 ('arranges', 1),
 ('availablity', 1),
 ('macus', 1),
 ('beet', 1),
 ('fraternity', 1),
 ('spotlesly', 1),
 ('guast', 1),
 ('vegfest', 1),
 ('deny', 1),
 ('wecolming', 1),
 ('strives', 1),
 ('morethe', 1),
 ('lovliness', 1),
 ('marucs', 1),
 ('esuite', 1),
 ('ridley', 1),
 ('alien', 1),
 ('prequel', 1),
 ('prometheus', 1),
 ('porriage', 1),
 ('britol', 1),
 ('banksies', 1),
 ('follies', 1),
 ('olra', 1),
 ('recomnded', 1),
 ('habla', 1),
 ('espanol', 1),
 ('sociability', 1),
 ('neighborhoody', 1),
 ('reorganized', 1),
 ('ohh', 1),
 ('arnofolni', 1),
 ('uniform', 1),
 ('exhudes', 1),
 ('assumption', 1),
 ('coexisting', 1),
 ('perfumes', 1),
 ('knocks', 1),
 ('huffy', 1),
 ('invasion', 1),
 ('cherish', 1),
 ('unpictured', 1),
 ('fernando', 1),
 ('eras', 1),
 ('tractor', 1),
 ('giuliana', 1),
 ('plumped', 1),
 ('londoners', 1),
 ('newington', 1),
 ('c

In [14]:
# Build the bigram & trigram models  [the model creates 2 or 3 wordy, most common occuring  phares]
bigram = gensim.models.Phrases(fitered_english_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[fitered_english_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[fitered_english_words[0]]])

['our', 'stay', 'with', 'marcus', 'in', 'bristol', 'was', 'fantastic', 'in', 'every', 'way', 'he', 'was', 'great', 'host', 'picking', 'us', 'up', 'at', 'the', 'bus', 'stop', 'recommending', 'places', 'to', 'try', 'leaving', 'plenty', 'of', 'pastries', 'and', 'other', 'breakfast', 'items', 'to', 'enjoy', 'in', 'the', 'morning', 'the', 'flat', 'itself', 'was', 'modern', 'bright', 'clean', 'and', 'spacious', 'and', 'best', 'of', 'all', 'right', 'on', 'bristol', 'lovely', 'harbourside', 'we', 'will', 'definitely', 'stay', 'again', 'next', 'time', 'we', 're', 'in', 'bristol', 'thanks', 'again', 'marcus']


In [15]:
from nltk.corpus import stopwords 
from nltk import pos_tag
stop_words = stopwords.words('english')
print('number of stop words:',len(stop_words))

# define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]



def get_nouns_adjectives(texts):
    ''' returns nouns(NN) and Adjectives from a the 2d lists(texts)'''
    data = [[ word_tag_pair[0] for word_tag_pair in pos_tag(sent) if (word_tag_pair[1]=='JJ' or word_tag_pair[1]=='NN')] for sent in texts]
    return data

    
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatize_texts(texts):
    lemmatizer = WordNetLemmatizer()
    data_out =[]
    for doc in texts:
            data_out.append([lemmatizer.lemmatize(word) for word in doc])
    return data_out 

number of stop words: 179


In [17]:
# remove the stopwords
data_words_nostops = remove_stopwords(fitered_english_words)
print("after removing stop words:",data_words_nostops[:2])

# get the only nouns and adjectives
noun_adj_words = get_nouns_adjectives(data_words_nostops)
print("\n\nOnly noun and adjective words:",noun_adj_words[:2])


after removing stop words: [['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'us', 'bus', 'stop', 'recommending', 'places', 'try', 'leaving', 'plenty', 'pastries', 'breakfast', 'items', 'enjoy', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'best', 'right', 'bristol', 'lovely', 'harbourside', 'definitely', 'stay', 'next', 'time', 'bristol', 'thanks', 'marcus'], ['marcus', 'brilliant', 'warm', 'friendly', 'host', 'picked', 'us', 'railway', 'station', 'took', 'anne', 'doctor', 'drove', 'us', 'around', 'wherever', 'needed', 'go', 'bristol', 'dropped', 'us', 'back', 'railway', 'station', 'leaving', 'flat', 'modern', 'comfortable', 'clean', 'well', 'heated', 'marcus', 'provided', 'us', 'everything', 'could', 'wish', 'wish', 'could', 'stayed', 'longer']]
 Only noun and adjective words: [['stay', 'fantastic', 'way', 'great', 'host', 'bus', 'plenty', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'right', 'bristol', 'harbourside'

In [18]:
# create bigrams
data_words_bigrams = make_bigrams(noun_adj_words)
print("\nbigram data words:",data_words_bigrams[0])

#  lemmatization
lemmatized_data = lemmatize_texts(data_words_bigrams)
print('\n\nAfter lemmatization:',lemmatized_data[0:2])


bigram data words: ['stay', 'fantastic', 'way', 'great', 'host', 'bus', 'plenty', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'right', 'bristol', 'harbourside', 'next', 'time', 'bristol', 'marcus']


After lemmatization: [['stay', 'fantastic', 'way', 'great', 'host', 'bus', 'plenty', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'right', 'bristol', 'harbourside', 'next', 'time', 'bristol', 'marcus'], ['marcus', 'brilliant', 'warm', 'picked', 'station', 'anne', 'doctor', 'wherever', 'bristol', 'railway_station', 'flat', 'modern', 'comfortable', 'clean', 'heated', 'marcus', 'everything', 'wish']]


In [19]:
# after removing stop words or pre-processing : check the most common and very the rare words
wf = get_words_frequency(lemmatized_data)
most_common_words = sorted(wf.items(), key=lambda x: x[1], reverse=True)
most_common_words[:20]   # first 20 words

[('great', 49955),
 ('place', 31728),
 ('stay', 30772),
 ('location', 30329),
 ('clean', 24612),
 ('room', 22558),
 ('bristol', 20884),
 ('host', 19193),
 ('house', 19121),
 ('comfortable', 18195),
 ('nice', 17885),
 ('good', 16611),
 ('flat', 15818),
 ('everything', 12855),
 ('home', 12424),
 ('apartment', 11695),
 ('city', 11453),
 ('easy', 10911),
 ('perfect', 10663),
 ('helpful', 9120)]

In [101]:
# after removing stop words or pre-processing : check the rare words
wf = get_words_frequency(lemmatized_data)
rare_words = sorted(wf.items(), key=lambda x: x[1])
rare_words_list =list(rare_words[:13500])
print(rare_words_list[:500])

[('chauffeur', 1), ('foreward', 1), ('boatload', 1), ('markus', 1), ('cellular', 1), ('saunter', 1), ('mastered', 1), ('availablity', 1), ('macus', 1), ('awaken', 1), ('beet', 1), ('fraternity', 1), ('guast', 1), ('deny', 1), ('morethe', 1), ('lovliness', 1), ('esuite', 1), ('impacted', 1), ('ridley', 1), ('alien', 1), ('prequel', 1), ('prometheus', 1), ('britol', 1), ('switzerland', 1), ('bedmister', 1), ('habla', 1), ('espanol', 1), ('sociability', 1), ('neighborhoody', 1), ('arnofolni', 1), ('uniform', 1), ('assumption', 1), ('perfumed', 1), ('huffy', 1), ('cherish', 1), ('fernando', 1), ('tractor', 1), ('newington', 1), ('cookie', 1), ('cutter', 1), ('vaya', 1), ('guapos', 1), ('airbn', 1), ('connoisseur', 1), ('mistery', 1), ('masaai', 1), ('samatha', 1), ('concentration', 1), ('collector', 1), ('twas', 1), ('candlelit', 1), ('prise', 1), ('bohemith', 1), ('thrift', 1), ('hazardous', 1), ('gaggle', 1), ('eccentricity', 1), ('trough', 1), ('older', 1), ('bastien', 1), ('samathas', 

In [28]:

def remove_rare_words(texts,words_freq):
    '''removes the rare words'''
    words_data = [[w for w in doc if words_freq[w]>7] for doc in texts]
    return words_data


In [31]:
# remove the rare words from texts
words_data = remove_rare_words(lemmatized_data,wf)
pprint(words_data[:50])

[['stay',
  'fantastic',
  'way',
  'great',
  'host',
  'bus',
  'plenty',
  'morning',
  'flat',
  'modern',
  'bright',
  'clean',
  'spacious',
  'right',
  'bristol',
  'harbourside',
  'next',
  'time',
  'bristol',
  'marcus'],
 ['marcus',
  'brilliant',
  'warm',
  'station',
  'anne',
  'doctor',
  'wherever',
  'bristol',
  'railway_station',
  'flat',
  'modern',
  'comfortable',
  'clean',
  'heated',
  'marcus',
  'everything',
  'wish'],
 ['mum',
  'angela',
  'marcus',
  'apartment',
  'bristol',
  'experience',
  'confortable',
  'nice',
  'area',
  'bristol',
  'close',
  'several',
  'great',
  'person',
  'needed',
  'generous',
  'practical',
  'help',
  'relocation',
  'company',
  'place',
  'special',
  'visit',
  'bristol'],
 ['marcus',
  'exceptional',
  'host',
  'night',
  'bristol',
  'flat',
  'location',
  'harbourside',
  'great',
  'stay'],
 ['marcus',
  'easy',
  'helpful',
  'informative',
  'place',
  'gorgeous',
  'everything',
  'great',
  'many',
 

In [32]:
# create dictionary
id2word = corpora.Dictionary(words_data)
print("number of keys:",len(id2word.keys()))
# view the first word of the dictionary
print(id2word[0])   # see word with key =0

# create corpus
corpus_texts= words_data
print(corpus_texts[:3])

# bag of words or vector spaces
corpus = [id2word.doc2bow(text) for text in corpus_texts]
print(corpus[:2])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

number of keys: 3973
bright
[['stay', 'fantastic', 'way', 'great', 'host', 'bus', 'plenty', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'right', 'bristol', 'harbourside', 'next', 'time', 'bristol', 'marcus'], ['marcus', 'brilliant', 'warm', 'station', 'anne', 'doctor', 'wherever', 'bristol', 'railway_station', 'flat', 'modern', 'comfortable', 'clean', 'heated', 'marcus', 'everything', 'wish'], ['mum', 'angela', 'marcus', 'apartment', 'bristol', 'experience', 'confortable', 'nice', 'area', 'bristol', 'close', 'several', 'great', 'person', 'needed', 'generous', 'practical', 'help', 'relocation', 'company', 'place', 'special', 'visit', 'bristol']]
[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(1, 1), (3, 1), (5, 1), (9, 2), (10, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]]


[[('bright', 1),
  ('bristol', 2),
  ('bus', 1),
  ('clean', 1),
  ('fantastic', 1),
  ('flat', 1),
  ('great', 1),
  ('harbourside', 1),
  ('host', 1),
  ('marcus', 1),
  ('modern', 1),
  ('morning', 1),
  ('next', 1),
  ('plenty', 1),
  ('right', 1),
  ('spacious', 1),
  ('stay', 1),
  ('time', 1),
  ('way', 1)]]

How does LDA work or converge?
`
For each document d, compute P( topic t | document d ) := proportion of words in document d that are assigned to topic t
For each topic t, P( word w | topic t ) := proportion of assignments to topic t that come from word w (across all documents)
For each word w, reassign topic t’, where we choose topic t’ with probability P( topic t’ | word w ) = P( topic t’ | document d ) * P( word w | topic t’ )`

In [33]:
import time
start_time = time.time()
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           chunksize=2000,
                                           passes=30,
                                           alpha='auto',
                                           per_word_topics=True)

done_time = time.time()

print(" %.3f secs" % (done_time - start_time))

331.863 secs


In [100]:
doc_lda=lda_model[corpus]
# doc_lda.corpus


In [75]:
print(" %.2f secs" % (done_time - start_time))

In [102]:
# Print top 10 keyword of each topic
(lda_model.print_topics())


[(0,
  '0.165*"stylish" + 0.099*"thank" + 0.083*"road" + 0.071*"weekend" + 0.070*"need" + 0.043*"parking" + 0.028*"nearby" + 0.024*"noise" + 0.023*"thing" + 0.022*"gloucester"'),
 (1,
  '0.142*"excellent" + 0.108*"value" + 0.084*"local" + 0.064*"money" + 0.044*"couple" + 0.039*"garden" + 0.034*"cozy" + 0.027*"anything" + 0.024*"pub" + 0.022*"sofa"'),
 (2,
  '0.086*"time" + 0.065*"check" + 0.039*"amazing" + 0.037*"next" + 0.032*"brilliant" + 0.032*"experience" + 0.031*"airbnb" + 0.030*"small" + 0.029*"visit" + 0.028*"book"'),
 (3,
  '0.143*"clean" + 0.092*"nice" + 0.086*"good" + 0.080*"comfortable" + 0.078*"room" + 0.040*"helpful" + 0.035*"quiet" + 0.035*"spacious" + 0.025*"bed" + 0.025*"cosy"'),
 (4,
  '0.078*"clifton" + 0.063*"coffee" + 0.039*"tea" + 0.032*"pleasant" + 0.031*"milk" + 0.025*"village" + 0.023*"cafe" + 0.023*"safe" + 0.022*"cute" + 0.021*"heart"'),
 (5,
  '0.155*"great" + 0.113*"place" + 0.096*"stay" + 0.093*"location" + 0.063*"host" + 0.054*"bristol" + 0.048*"flat" + 0.

In [80]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel

lsi_model = LsiModel(corpus=corpus,id2word=id2word,num_topics=15,chunksize=2000,power_iters=30)  # train model


# model.add_documents(common_corpus[4:])  # update model with new documents
# tmp_fname = get_tmpfile("lsi.model")
# model.save(tmp_fname)  # save model
# loaded_model = LsiModel.load(tmp_fname)  # load model

In [81]:
lsi_model.print_topics()

[(0,
  '0.523*"great" + 0.295*"place" + 0.294*"stay" + 0.271*"location" + 0.211*"clean" + 0.210*"room" + 0.207*"bristol" + 0.178*"house" + 0.161*"comfortable" + 0.159*"host"'),
 (1,
  '-0.737*"great" + 0.280*"room" + 0.244*"house" + -0.224*"location" + 0.189*"nice" + 0.180*"stay" + 0.166*"clean" + 0.157*"comfortable" + 0.150*"place" + 0.145*"bristol"'),
 (2,
  '-0.761*"place" + -0.352*"stay" + 0.338*"room" + 0.290*"house" + 0.171*"location" + 0.099*"comfortable" + 0.092*"clean" + 0.081*"good" + 0.061*"flat" + 0.057*"host"'),
 (3,
  '0.566*"flat" + -0.364*"house" + -0.343*"room" + 0.339*"location" + -0.301*"great" + 0.209*"apartment" + 0.195*"bristol" + 0.160*"good" + -0.123*"place" + 0.113*"clean"'),
 (4,
  '0.766*"stay" + -0.449*"place" + -0.242*"nice" + -0.211*"good" + -0.149*"location" + -0.133*"clean" + -0.124*"room" + 0.105*"home" + 0.087*"flat" + 0.071*"comfortable"'),
 (5,
  '-0.560*"house" + 0.470*"room" + -0.459*"bristol" + 0.237*"stay" + 0.194*"clean" + 0.150*"comfortable" + 

` NB: next steps (a)interpretation of the topics,(b)visualizing the topics clusters and documents ranking...`
--------------------
`references`: gensim documentation 